In [ ]:
import pandas as pd
import os
import plotly.express as px
os.chdir(r'D:\Projects\football-odds-analysis')
from utils.save_as_video import plotly_fig_to_video
from utils.preprocess import preprocess_pipeline
from utils.margins import get_avg_margin, get_avg_ballanced_unbalanced_margin
from input_variables import DIVISIONS

bookmakers_data = preprocess_pipeline('all_avail_games.csv')
avg_margin_data = pd.DataFrame()

for key in bookmakers_data.keys():
    if key != 'AVG':
        avg_margin_data = pd.concat([avg_margin_data, get_avg_margin(bookmakers_data[key],
                                                                     key)], axis=0)
    else:
        avg_margin_data = pd.concat([avg_margin_data, get_avg_margin(bookmakers_data[key], 
                                                                     key, 'Avg_return_on_game')], axis=0)

avg_margin_data = avg_margin_data.loc[:,~avg_margin_data.columns.duplicated()]
avg_margin_data.columns = ['League', 'Year', 'Average Margin of Games', 'Bookmaker']
avg_margin_data['League Name'] = avg_margin_data['League'].map(DIVISIONS)

In [ ]:
fig = px.scatter(avg_margin_data, x="League Name", y="Average Margin of Games", animation_frame="Year",
                 color="Bookmaker", range_y=[0.01,0.2])
fig.update_traces(marker_size=10)
fig.update_layout(
    autosize=False,
    width=900,
    height=500,)
fig

In [ ]:
UNBALANCE_THRESHOLD = 0.8

# Balanced Margins
avg_margin_data_balanced = pd.DataFrame()
for key in bookmakers_data.keys():
    if key != 'AVG':
        avg_margin_data_balanced = pd.concat([avg_margin_data_balanced, get_avg_ballanced_unbalanced_margin(bookmakers_data[key],
                                                                                                            key, 
                                                                                                            'Balanced', 
                                                                                                            UNBALANCE_THRESHOLD)], axis=0)

avg_margin_data_balanced = avg_margin_data_balanced.loc[:,~avg_margin_data_balanced.columns.duplicated()]
avg_margin_data_balanced.columns = ['League', 'Year', 'Average Margin of Games With Balanced Probability', 'Bookmaker']
avg_margin_data_balanced['League Name'] = avg_margin_data_balanced['League'].map(DIVISIONS)

# Unbalanced Margins
avg_margin_data_unbalanced = pd.DataFrame()
for key in bookmakers_data.keys():
    if key != 'AVG':
        avg_margin_data_unbalanced = pd.concat([avg_margin_data_unbalanced, get_avg_ballanced_unbalanced_margin(bookmakers_data[key],
                                                                                                                key, 
                                                                                                                'Unbalanced', 
                                                                                                                UNBALANCE_THRESHOLD)], axis=0)

avg_margin_data_unbalanced = avg_margin_data_unbalanced.loc[:,~avg_margin_data_unbalanced.columns.duplicated()]
avg_margin_data_unbalanced.columns = ['League', 'Year', 'Average Margin of Games With Unbalanced Probability', 'Bookmaker']
avg_margin_data_unbalanced['League Name'] = avg_margin_data_unbalanced['League'].map(DIVISIONS)

In [ ]:
fig = px.scatter(avg_margin_data_balanced, x="League Name", y="Average Margin of Games With Balanced Probability", animation_frame="Year",
                 color="Bookmaker", range_y=[0.01,0.2])
fig.update_traces(marker_size=10)
fig.update_layout(
    autosize=False,
    width=900,
    height=500,)
fig

In [ ]:
fig = px.scatter(avg_margin_data_unbalanced, x="League Name", y="Average Margin of Games With Unbalanced Probability", animation_frame="Year",
                 color="Bookmaker", range_y=[0.01,0.2])
fig.update_traces(marker_size=10)
fig.update_layout(
    autosize=False,
    width=900,
    height=500,)
fig